## Code Review 2
## Week 2
## Exercise 4: Mining of Massive Datasets Text

### Exercise 3.1.1

In [1]:
s1 = {1, 2, 3, 4}
s2 = {2, 3, 5, 7}
s3 = {2, 4, 6}

In [2]:
def jaccard_sim (x, y):
    intx = len(x & y)
    union = len(x | y)
    jacard_sim = intx/union
    return jacard_sim

Jaccard Similarity of set 1 and set 2

In [3]:
jaccard_sim(s1, s2)

0.3333333333333333

Jaccard Similarity of set 2 and set 3

In [4]:
jaccard_sim(s2, s3)

0.16666666666666666

Jaccard Similarity of set 1 and set 3

In [5]:
jaccard_sim(s1, s3)

0.4

Jaccard Similarity of all three sets

In [6]:
intx = len(s1 & s2 & s3)
union = len(s1 | s2 | s3)

In [7]:
intx/union

0.14285714285714285

### Exercise 3.2.1

In [8]:
words = 'The most effective way to represent documents as sets for the purpose of identifying lexically similar documents is to construct from the document the set of short strings that appear within it.'

In [9]:
# function to create k-shingles based on individual words and n output

def shingles_words(string, k, n):      # k length shingles  n output
    string = string.split(' ')          # split string on white space
    shingles = []
    for i in range(len(string[:n+2])-k+1):    # range in length of string or n plus k + 1
        s = ' '.join(string[i:i+k])     # join k words back together for each word through word plus k
        shingles.append(s)              # append s to shingles
    return shingles

In [10]:
shingles_words(words, 3, 10)

['The most effective',
 'most effective way',
 'effective way to',
 'way to represent',
 'to represent documents',
 'represent documents as',
 'documents as sets',
 'as sets for',
 'sets for the',
 'for the purpose']

In [11]:
# function to create k-shingles based on characters and n output

def shingles_char(string, k, n):             # k length shingles   n output
    shingles = []
    for i in range(len(string[:n+2])-k+1):   # range in length of string or n plus k+1 
        shingles.append(string[i:i+k])       # append letter through k to shingles list 
    return shingles

In [12]:
shingles_char(words, 3, 10)

['The', 'he ', 'e m', ' mo', 'mos', 'ost', 'st ', 't e', ' ef', 'eff']

### Exercise 3.3.3

In [13]:
import pandas as pd
import sys
import numpy as np

#### (A)

Matrix

In [14]:
matrix = np.array([[0, 1, 0, 1],
          [0, 1, 0, 0],
          [1, 0, 0, 1],
          [0, 0, 1, 0],
          [0, 0, 1, 1],
          [1, 0, 0, 0]])

Hash Functions

In [15]:
def h1(r):
    h1 = (2*r + 1) % 6
    return h1

In [16]:
def h2(r):
    h2 = (3*r + 2) % 6
    return h2

In [17]:
def h3(r):
    h3 = (5*r + 2) % 6
    return h3

Minhash Signiture

In [18]:
def minhash_sig_matrix(matrix, hash_functions):
    rows = len(matrix)           # number of rows
    cols = len(matrix[0])        # number of columns - length of first array
    n_hash = len(hash_functions)   # number of hash functions 

    minhash_sig_matrix = []         # create empty signature matrix
    for i in range(n_hash):   # for each n hash function
        minhash_sig_matrix.append([sys.maxsize] * cols)   # add infinity with maxsize * num of columns to sig matrix
    
    minhash_sig_matrix = np.asarray(minhash_sig_matrix) # convert sig matrix list to np matrix
    
    for r in range(rows): 
        hashfunc = list(map(lambda x: x(r), hash_functions)) # for each hash function map it to each row r
        # if data != 0 and signature > hash value, replace signature with hash value
        for c in range(cols):
            if matrix[r,c] == 1:        # if value in matrix is 1 
                for i in range(n_hash):
                    if hashfunc[i] < minhash_sig_matrix[i,c]:  # if hashfunc is smaller than value in matrix
                        minhash_sig_matrix[i,c] = hashfunc[i]  # then replace value in matrix with hashfunc

        minhash_sig_df = pd.DataFrame(minhash_sig_matrix, columns=['S1', 'S2', 'S3', 'S4'], index=['h1','h2','h3'])
        
    return minhash_sig_df

In [19]:
minhash_sig_matrix(matrix, [h1, h2, h3])

,S1,S2,S3,S4
h1,5,1,1,1
h2,2,2,2,2
h3,0,1,4,0


#### (B)

In [20]:
# function to compute the values of each hash function to the rows
# creates permutation of each row if it were a hash function

def hash_permutation(matrix, hash_functions):
    rows = len(matrix)           # number of rows
    n_hash = len(hash_functions)   # number of hash functions 
    
    hash_func = []
    
    for r in range(rows): 
        # map each hash function to the row #
        compute_hash = list(map(lambda x: x(r), hash_functions))
        # append to hash_func list each value
        hash_func.append(compute_hash)
    
    # create dataframe of the hash functions and their rows
    hash_func_df = pd.DataFrame(hash_func, columns=['h1', 'h2', 'h3'])
    
    return hash_func_df

In [21]:
hash_permutation(matrix, [h1, h2, h3])

,h1,h2,h3
0,1,2,2
1,3,5,1
2,5,2,0
3,1,5,5
4,3,2,4
5,5,5,3


When applying each hash function to each row for each sig, it results that h3 is a true permutation and h1 and h2 are not. This is because in h3, all rows have a different number, 0-5. There are no repeats. For h1 and h2, there are repeats of the same hash value for some of the rows. In h1, row 0 and row 3 have the same hash of 1. Row 1 and row 4 have the same hash of 3. Row 2 and row 5 have the same hash of 5. In h2, multiple rows have the same hash.

#### (C)

#### Set 1 vs. Set2


estimated: .33

true jaccard: 0

These are close but different.

#### Set 1 vs. Set 3

estimated: .33

true jaccard: 0

These are close but different.

#### Set 1 vs. Set 4

estimated: .66

true jaccard: .33

These are close but different.

#### Set 2 vs. Set 3

estimated: .66

true jaccard: 0

These not similar.

#### Set 2 vs. Set 4

estimated: .66

true jaccard: .25

These not similar.

#### Set 3 vs. Set 4

estimated: .66

true jaccard: .25

These are not similar.

### Exercise 3.4.1

In [22]:
# probability that the signatures agree in all the rows of at least one band

def s_curve(s, r, b):
    output = []
    for i in s:
        output.append((1- (1 - (i**r))**b))
        
    return output

In [23]:
s = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

#### rows: 3 band:10

In [24]:
s_curve(s, 3, 10)

[0.009955119790251765,
 0.07718058804273675,
 0.23944889319887064,
 0.4838707317677322,
 0.7369244238361716,
 0.9122674753991765,
 0.985015105295655,
 0.9992340538808936,
 0.9999978635491371]

The signatures start to agree in all the rows of at least one band where jaccard similarity is 0.07.



#### rows: 6  band:20

In [25]:
s_curve(s, 6, 20)

[1.9999810001669616e-05,
 0.001279222058761964,
 0.014479466504172311,
 0.07880932311056232,
 0.27018714400947597,
 0.6154146360312677,
 0.9181859965846741,
 0.9977121251546806,
 0.9999997398129465]

The signatures start to agree in all the rows of at least one band where jaccard similarity is 0.08.

#### rows:5 band: 50

In [26]:
s_curve(s, 5, 50)

[0.0004998775195954597,
 0.01587519984502117,
 0.11453988231042189,
 0.4022839522088044,
 0.7955506304323648,
 0.9825338277068608,
 0.9998989958361557,
 0.9999999976077777,
 1.0]

The signatures start to agree in all the rows of at least one band where jaccard similarity is 0.06.